# Wind Turbine Object Detection from Aerial Imagery Using TensorFlow Object Detection API and Google Colab

## Introduction

This notebook provides the full pipeline to perform training and inference for a wind turbine object detection model using publicly available aerial images and the [TensorFlow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection). It is designed to run in [Google Colab](https://colab.research.google.com/notebooks/welcome.ipynb), a Jupyter notebook environment running on a virtual machine (VM) that provides free access to a Tesla K80 GPU for up to 12 hours.


The aerial image data set used in this notebook is obtained from the [National Agriculture Imagery Program (NAIP) database](https://www.fsa.usda.gov/programs-and-services/aerial-photography/imagery-programs/naip-imagery/) using [USGS EarthExplorer](https://earthexplorer.usgs.gov/). The particular NAIP images used to train, test, and validate this model are from three wind farms located in west-central Iowa containing turbines of varying capacity, style, and manufacturer. A sample NAIP image is presented below in the "Sample NAIP image" section. The original NAIP images are 5978 x 7648 so they had to be chipped into smaller individual images to avoid excessive memory use. In addition, the ratio of object size to image size is improved by this operation. An image size of 300 x 300 was chosen since the TensorFlow object detection SSD-based models rescale all input images to this size. 

A total of 488 images, all containing at least one full wind turbine, were collected and split into train (\~80%), test (\~16%), and validate (\~4%) sets. [LabelImg](https://github.com/tzutalin/labelImg) was then used to label all the images in the train and test sets. Samples of the chipped and annotated images are shown below in the "Sample chipped and annotated NAIP images" section. Annotating the images in LabelImg creates an XML file corresponding to each image. These XML files must be converted to CSV and then TFRecords. Sample code for this can be found [here](https://towardsdatascience.com/how-to-train-your-own-object-detector-with-tensorflows-object-detector-api-bec72ecfe1d9) or [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html) (among other places).

In [0]:
# clone wind-turbine-detector repo
!git clone https://github.com/lbborkowski/wind-turbine-detector.git

### Sample NAIP image

In [0]:
from matplotlib import pyplot as plt
from PIL import Image
import os
import glob

%matplotlib inline

image = Image.open('/content/wind-turbine-detector/images/samples/orig/m_4109442_se_15_1_20170709.jpg')
plt.figure(figsize=(12,8))
plt.axis('off')
plt.imshow(image)

### Sample chipped and annotated NAIP images

In [0]:
PATH_TO_SAMPLE_IMAGES_DIR = '/content/wind-turbine-detector/images/samples/chopped'
SAMPLE_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_SAMPLE_IMAGES_DIR, "*.*"))

for image_path in SAMPLE_IMAGE_PATHS:
  image = Image.open(image_path)
  plt.figure(figsize=(8,8))
  plt.axis('off')
  plt.imshow(image)

## Training
Training will be performed on the 392 labeled images in the train image set and tested against the 80 labeled test images. A pre-trained model from the [TensorFlow Object Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) is used as a starting point. In this notebook, the ssd_inception_v2_coco model is used based on its balance of accuracy and efficiency.  


> *Note: Training will take ~2.5 hours. If you wish to bypass the training step, you can skip the "Train model" and "Export trained wind turbine detector model" sections and uncomment the second "PATH_TO_FROZEN_GRAPH=" line in the "Inference" section to use the provided pre-trained wind turbine detection model.*



### Install all required libraries
Further details on how to install and configure TensorFlow Object Detection API can be found [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md).

In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install contextlib2
!pip install jupyter
!pip install matplotlib

### Clone TensorFlow Object Detection API repo

In [0]:
!git clone --quiet https://github.com/tensorflow/models.git

### COCO API installation
This is needed if you are interested in using COCO evaluation metrics.

In [0]:
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI; make; cp -r pycocotools /content/models/research/

### Protobuf compilation
The Protobuf libraries provided in the TensorFlow Object Detection API repo must be compiled in order to use the framework.

In [0]:
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

### Add Libraries to PYTHONPATH

In [0]:
%set_env PYTHONPATH=$PYTHONPATH:/content/models/research:/content/models/research/slim

### Test the installation

In [0]:
!python object_detection/builders/model_builder_test.py

### Setup and run TensorBoard
TensorBoard provides a visualization of various quantitative metrics such as loss as well as a comparison between prediction vs. ground truth for a subset of images.

In [0]:
%cd /content/wind-turbine-detector

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

In [0]:
LOG_DIR = 'training/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

#### Get TensorBoard link
Click on the link to launch TensorBoard. It will update once the first checkpoint is saved. The plot of the "loss_1" scalar will provide the loss as a function of step, matching what is printed to the screen.

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Train model
Train the wind turbine detector model using a modified model_main.py file which includes "tf.logging.set_verbosity(tf.logging.INFO)" following the import statements to output the loss every 100 steps. The model configuration is provided in wind-turbine-detector/training/ssd_inception_v2_coco_WTDetector.config. This configuration file uses all the default settings provided in the sample ssd_inception_v2_coco.config file except the following:

*   num_classes: 1
*   batch_size: 12
*   fine_tune_checkpoint: "pre-trained-model/model.ckpt"
*   train_input_reader: {
  tf_record_input_reader {
    input_path: "annotations/train.record"
  }
  label_map_path: "annotations/label_map.pbtxt"
}
    * *Note: The 'label_map.pbtxt' file required for training contains 1 class: item {
id: 1
name: 'wind turbine'
}*
*   eval_input_reader: {
  tf_record_input_reader {
    input_path: "annotations/test.record"
  }
  label_map_path: "annotations/label_map.pbtxt"
  shuffle: false
  num_readers: 1
}  



Additional data (image) augmentation was prescribed in the configuration file. Combining a vertical flip and a 90 degree rotation with the default horizontal flip, the training data can be extended to contain all possible wind turbine orientations. These operations help to generalize the model.
*   data_augmentation_options {
    random_vertical_flip {
    }
  }
*   data_augmentation_options {
    random_rotation90 {
    }
  }










In [0]:
!python model_main.py --pipeline_config_path=training/ssd_inception_v2_coco_WTDetector.config --model_dir=training/ --num_train_steps=20000 --alsologtostderr
#!python train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/ssd_inception_v2_coco_WTDetector.config # using legacy training code

/content/models/research/object_detection/utils/visualization_utils.py:29: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "model_main.py", line 26, in <module>
    from object_detection import model_lib
  File "/content/models/research/object_detection/model_lib.py", line 27, in <module>
    from object_detection import eval_util
  File "/content/models/research/object_detection/eval_util.py", line 33, in <module>
    from object_detection.metrics import coco_evaluation
  File "/content/models/research/object_detection/metrics/coco_evaluation.py", line 25, in <module>
    from object_detection.metrics import coco_tools
  File "/content/models/research/object_detection/metrics/coco_tools.py", line 51, in 

### Export trained wind turbine detector model

In [0]:
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=training/ssd_inception_v2_coco_WTDetector.config \
    --output_directory=WTDetectorModel \
    --trained_checkpoint_prefix=training/model.ckpt-20000

## *Inference*

Perform inference using the newly trained wind turbine detection model on the validation image set. This set of images was kept separate from the test and train image sets and will now be used to validate the accuracy of the model.   

In [0]:
%cd /content/models/research/object_detection

### Imports

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

#if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
#  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

#### Env setup

In [0]:
# This is needed to display the images.
%matplotlib inline

#### Object detection imports
Here are the imports from the object detection module.

In [0]:
from utils import label_map_util

from utils import visualization_utils as vis_util

### Model preparation 

#### Variables

Any model exported using the "export_inference_graph.py" tool can be loaded here simply by changing "PATH_TO_FROZEN_GRAPH" to point to a new .pb file.  

In [0]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = '/content/wind-turbine-detector/WTDetectorModel/frozen_inference_graph.pb'
#PATH_TO_FROZEN_GRAPH = '/content/wind-turbine-detector/trainedWTDetector/frozen_inference_graph.pb' # Uncomment this line to run inference (without training) using provided pre-trained model

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/content/wind-turbine-detector/annotations/label_map.pbtxt'

#### Load a (frozen) TensorFlow model into memory

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

#### Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to "airplane".  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [0]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
print(category_index)

#### Helper code

In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

#### Inference function
Perform inference one image at a time

In [0]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

### Detection - Validation Images
Detect wind turbines in validation image set. 

This section provides the validation images containing at least one wind turbine and corresponding bounding box(es).

In [0]:
import os
import glob

PATH_TO_TEST_IMAGES_DIR = '/content/wind-turbine-detector/images/valid'
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
print(TEST_IMAGE_PATHS)

# Size, in inches, of the output images.
IMAGE_SIZE = (6, 6)

#### Perform inference on all validation images
Loop over validation set and output images containing detected wind turbine with its location denoted with a bounding box.

In [0]:
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.axis('off')
  plt.imshow(image_np)

### Detection - Full NAIP Images
Detect wind turbines in full size NAIP images (~4 miles square) using sliding window of size 300 x 300 pixels over the entire 5978 x 7648 pixel image. 

This section outputs the individual images containing at least one wind turbine and corresponding bounding box(es), the full NAIP image with all the wind turbines marked, and the latitude and longitude coordinates of all detected wind turbines. 

In [0]:
import os
import glob

PATH_TO_TEST_IMAGES_DIR = '/content/wind-turbine-detector/images/samples/orig'
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.jpg"))
print(TEST_IMAGE_PATHS)

# Size, in inches, of the output images.
IMAGE_SIZE = (8, 8)

#### Perform inference on full NAIP images
For each full NAIP image, perform inference on sliding 300 x 300 pixel window. Output images containing detected wind turbine with its location denoted with a bounding box.

In [0]:
BBsWTs = np.empty((0, 4))
detectedWTs=[]
chipsize = 300
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  width, height = image.size
  ii=0
  for x0 in range(0, width, chipsize): # width
    ii+=1
    jj=0
    for y0 in range(0, height, chipsize): # height
      jj+=1
      box = (x0, y0,
             x0+chipsize,
             y0+chipsize)
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      image_np = load_image_into_numpy_array(image.crop(box))
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Actual detection.
      output_dict = run_inference_for_single_image(image_np, detection_graph)
      BBsWTs=np.append(BBsWTs,output_dict['detection_boxes'][output_dict['detection_scores']>0.5],axis=0)
      if len(output_dict['detection_scores'][output_dict['detection_scores']>0.5])>0:
        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
            category_index,
            instance_masks=output_dict.get('detection_masks'),
            use_normalized_coordinates=True,
            line_thickness=8)
        plt.figure(figsize=IMAGE_SIZE)
        plt.axis('off')
        for kk in range(len(output_dict['detection_scores'][output_dict['detection_scores']>0.5])):
          plt.plot(chipsize*np.mean([BBsWTs[-1-kk][1],BBsWTs[-1-kk][3]]),chipsize*np.mean([[BBsWTs[-1-kk][0],BBsWTs[-1-kk][2]]]),'bo')
          detectedWTs.append([image_path,ii,jj])
        plt.imshow(image_np)

In [0]:
# total number of wind turbines detected 
print(len(detectedWTs))
print(detectedWTs)

####Bounding box center location
Calculate normalized coordinates of bounding box center. This will be used to plot each detected wind turbine on the NAIP images and to compute the latitude and longitude of each wind turbine.

In [0]:
centerBBs = np.empty((len(BBsWTs),2))
ii=0
for BBs in BBsWTs:
  centerBBs[ii][:]=[np.mean([BBs[1],BBs[3]]),np.mean([[BBs[0],BBs[2]]])]
  ii+=1

####Plot wind turbine locations over NAIP image
Plot marker for each detected wind turbine on NAIP images analyzed.

In [0]:
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  plt.figure(figsize=(width/500,height/500))
  plt.axis('off')
  for ll in range(len(detectedWTs)):
    if detectedWTs[ll][0]==image_path:
      x_plot=chipsize*(detectedWTs[ll][1]-1+centerBBs[ll][0])
      y_plot=chipsize*(detectedWTs[ll][2]-1+centerBBs[ll][1])
      plt.plot(x_plot,y_plot,'ro')
  plt.imshow(image)


####Calculate wind turbine latitude and longitude
Perform a series of 1D linear interpolations to determine latitude and longitude coordinates of each detected wind turbine.

In [0]:
# initialize dictionaries for NAIP image corners
#   y1,x1 are lat,long of upper left corner of image
#   y2,x2 are lat,long of upper right corner of image
#   y3,x3 are lat,long of lower right corner of image
#   y4,x4 are lat,long of lower left corner of image
x1={'/content/wind-turbine-detector/images/samples/orig/m_4109442_se_15_1_20170709.jpg':-94.8178888,
   '/content/wind-turbine-detector/images/samples/orig/m_4209426_ne_15_1_20170707.jpg':-94.8180499}
y1={'/content/wind-turbine-detector/images/samples/orig/m_4109442_se_15_1_20170709.jpg':41.3151166,
   '/content/wind-turbine-detector/images/samples/orig/m_4209426_ne_15_1_20170707.jpg':42.6276193}
x2={'/content/wind-turbine-detector/images/samples/orig/m_4109442_se_15_1_20170709.jpg':-94.7464999,
   '/content/wind-turbine-detector/images/samples/orig/m_4209426_ne_15_1_20170707.jpg':-94.7464221}
y2={'/content/wind-turbine-detector/images/samples/orig/m_4109442_se_15_1_20170709.jpg':41.3162222,
   '/content/wind-turbine-detector/images/samples/orig/m_4209426_ne_15_1_20170707.jpg':42.6287332}
x3={'/content/wind-turbine-detector/images/samples/orig/m_4109442_se_15_1_20170709.jpg':-94.7446638,
   '/content/wind-turbine-detector/images/samples/orig/m_4209426_ne_15_1_20170707.jpg':-94.7444999}
y3={'/content/wind-turbine-detector/images/samples/orig/m_4109442_se_15_1_20170709.jpg':41.2473638,
   '/content/wind-turbine-detector/images/samples/orig/m_4209426_ne_15_1_20170707.jpg':42.5598722}
x4={'/content/wind-turbine-detector/images/samples/orig/m_4109442_se_15_1_20170709.jpg':-94.8159777,
   '/content/wind-turbine-detector/images/samples/orig/m_4209426_ne_15_1_20170707.jpg':-94.8160472}
y4={'/content/wind-turbine-detector/images/samples/orig/m_4109442_se_15_1_20170709.jpg':41.246261,
   '/content/wind-turbine-detector/images/samples/orig/m_4209426_ne_15_1_20170707.jpg':42.5587611}

# loop over all the detected wind turbines and calculate the 
# latitude and longitude coordinates based on its location in
# full NAIP images with respect to the image corners
latlongWTs=[]
for ll in range(len(detectedWTs)):
  a5=chipsize*(detectedWTs[ll][1]-1+centerBBs[ll][0])
  b5=chipsize*(detectedWTs[ll][2]-1+centerBBs[ll][1])

  p_a5=a5/width
  p_b5=b5/height

  x_le=x1[detectedWTs[ll][0]]+p_b5*(x4[detectedWTs[ll][0]]-x1[detectedWTs[ll][0]])
  x_re=x2[detectedWTs[ll][0]]+p_b5*(x3[detectedWTs[ll][0]]-x2[detectedWTs[ll][0]])
  x_WT=x_le+p_a5*(x_re-x_le)

  y_te=y1[detectedWTs[ll][0]]+p_a5*(y2[detectedWTs[ll][0]]-y1[detectedWTs[ll][0]])
  y_be=y4[detectedWTs[ll][0]]+p_a5*(y3[detectedWTs[ll][0]]-y4[detectedWTs[ll][0]])
  y_WT=y_te+p_b5*(y_be-y_te)
  latlongWTs.append([y_WT,x_WT])

# print out latitude and longitude coordinates for verification or plotting
print(latlongWTs)

## Summary

The trained model accurately detects at least 15 out of 17 wind turbines in the validation image set with high probability. This represents an accuracy of ~90%. Higher accuracy would likely be achieved by using a larger set of images (train + test) as well as using a more accurate pre-trained model. Alternative models, including those with higher mAP, can be found at the [TensorFlow Object Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md). Details of the trade-offs between speed, accuracy, and memory for various object detection model architectures (Faster RCNN, SSD, R-FCN) can be found in this [paper](https://arxiv.org/pdf/1611.10012.pdf), which can serve as a good starting point in determining which architecture is best for your application.

Using the trained model, images encompassing a large area (~16 sq miles) were scanned for wind turbines. Once detected, the position of each wind turbine was plotted on the original full NAIP image and its position (latitude, longitude) was output. The comprehensive pipeline including training, validation, and inference could be applied to many other applications involving detection of objects from aerial or satellite images. 